Code by Xiaorui Yang

2023.08.11

1. Random angle phi
2. Random starting position (the probability is proportional to the number of collisions the participant)

In [111]:
import pandas as pd
import numpy as np
import math
from shapely.geometry import LineString, Point
import matplotlib.pyplot as plt
import random



In [6]:
filename = 'Glauber_Pb_Pb_67_75000_v2 (1).txt'
data = pd.read_csv(filename, delimiter= '\s+', names=["Nucleus", "X-Coordinate", "Y-Coordinate", "Z-Coordinate", "Collision Number", "XXX", "XX"])
data

75000 Angles & Starting Positions

In [ ]:

data['Nucleus'] = pd.to_numeric(data['Nucleus'], errors='coerce')
data_collide = data[data['Collision Number'] > 0]


radius = 0.75
num_intersect_minus = []
num_intersect_plus = []
circles, event_num = [], []
num_part = []
epsilon_2, epsilon_3, psi_2, psi_3, imp_prmtr = [], [], [], [], []
length_output, start_x, start_y, collisionN_output, angle_output = [], [], [], [], []
length_arr = []
event_data = []


def process_event_data(event_data, length_arr):
    if event_data:

        length = length_arr[-2]

        current_event_df = pd.DataFrame(event_data[:-1]) 
        collision_numbers = current_event_df['Collision Number'].values
        probabilities = collision_numbers / collision_numbers.sum()
        
        selected_row = current_event_df.sample(n=1, weights=probabilities)


        start_coords_X = selected_row['X-Coordinate']
        start_coords_Y = selected_row['Y-Coordinate']
        num_c = selected_row['Collision Number']

        phi = int(random.uniform(0, 180))
        while phi == 180:
            phi = int(random.uniform(0, 180))
        
        phi_rad = math.radians(phi)


        angle_output.append(phi)
        start_x.append(start_coords_X)
        start_y.append(start_coords_Y)
        collisionN_output.append(num_c)

        # +L
        x_end_above = start_coords_X + length * math.cos(phi_rad)
        y_end_above = start_coords_Y + length * math.sin(phi_rad)

        line = LineString([(start_coords_X, start_coords_Y), (x_end_above, y_end_above)])
        count = sum(line.distance(Point(x, y)) < radius for x, y in circles)
        num_intersect_plus.append(count)

        # -L
        x_end_below = start_coords_X - length * math.cos(phi_rad)
        y_end_below = start_coords_Y - length * math.sin(phi_rad)

        line = LineString([(start_coords_X, start_coords_Y), (x_end_below, y_end_below)])
        count = sum(line.distance(Point(x, y)) < radius for x, y in circles)
        num_intersect_minus.append(count)


for index, row in data_collide.iterrows():
    if pd.notna(row['Nucleus']):

        length_arr.append(row['X-Coordinate'])  

        process_event_data(event_data, length_arr)
        
        event_num.append(int(row['Nucleus']))
        num_part.append(row['X-Coordinate'])
        epsilon_2.append(row['Y-Coordinate'])
        epsilon_3.append(row['Z-Coordinate'])
        psi_2.append(row['Collision Number'])
        psi_3.append(row['XXX'])
        imp_prmtr.append(row['XX'])

        event_data.clear()
        circles.clear()
    event_data.append(row.to_dict())
    circles.append((row['X-Coordinate'], row['Y-Coordinate']))


if event_data:
    process_event_data(event_data, length_arr)


